<a href="https://colab.research.google.com/github/wizkers007/fooddetection/blob/main/MiDaS_Test_Run_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install torch torchvision torchaudio opencv-python matplotlib
!pip install git+https://github.com/ultralytics/yolov5.git

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 40.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 32.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 61.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.7/188.7 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 89.1 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalli

In [ ]:
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 27.4 MB/s eta 0:00:00


In [ ]:
import torch
import cv2
import numpy as np
import matplotlib.pyplot as plt

# --- Step 1: Load MiDaS depth model ---
midas = torch.hub.load("intel-isl/MiDaS", "DPT_Large")
midas.to("cuda" if torch.cuda.is_available() else "cpu").eval()
midas_transforms = torch.hub.load("intel-isl/MiDaS", "transforms")
transform = midas_transforms.dpt_transform

# --- Step 2: Load YOLOv5 for spoon detection ---
yolo_model = torch.hub.load('ultralytics/yolov5', 'yolov5s', pretrained=True)
#yolo_model.classes = [43]  # class 43 is 'spoon' in COCO



# --- Load image ---
from google.colab import files
uploaded = files.upload()
img_path = list(uploaded.keys())[0]
img = cv2.imread(img_path)
img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

# --- Step 3: Get MiDaS depth map ---
input_batch = transform(img_rgb).to("cuda" if torch.cuda.is_available() else "cpu")
with torch.no_grad():
    prediction = midas(input_batch)
    prediction = torch.nn.functional.interpolate(
        prediction.unsqueeze(1),
        size=img_rgb.shape[:2],
        mode="bicubic",
        align_corners=False
    ).squeeze()
depth_map = prediction.cpu().numpy()

# --- Step 4: Detect spoon using YOLOv5 ---
results = yolo_model(img_rgb)
detections = results.pandas().xyxy[0]


# Filter for spoon class (COCO class 43)
spoon_detections = detections[detections['class'] == 43]
spoon = None

if not spoon_detections.empty:
    spoon = spoon_detections.iloc[0]
    x1, y1, x2, y2 = map(int, [spoon['xmin'], spoon['ymin'], spoon['xmax'], spoon['ymax']])
    cv2.rectangle(img_rgb, (x1, y1), (x2, y2), (0, 255, 0), 2)
    spoon_length_px = np.linalg.norm([x2 - x1, y2 - y1])
    print(f"Spoon pixel length: {spoon_length_px:.2f}")
else:
    print("Spoon not detected.")
    exit()

# --- Step 5: Estimate scale (depth unit to cm) ---
REAL_SPOON_LENGTH_CM = 15  # Assume average spoon is 15 cm
scale = REAL_SPOON_LENGTH_CM / spoon_length_px  # cm per pixel

########################Okay upto this point##############################

# --- Step 6: Estimate food height relative to plate ---
# Assume the plate is the region surrounding the spoon, use min depth near edges
plate_region = depth_map[y2+10:y2+30, x1-10:x2+10]  # below spoon
plate_depth = np.median(plate_region)

# Assume food is the region above the spoon (center of plate)
food_region = depth_map[y1-80:y1, x1:x2]
food_depth = np.median(food_region)

# --- Step 7: Calculate relative food height ---
relative_depth = plate_depth - food_depth
food_height_cm = relative_depth * scale

print(f"Estimated food height: {food_height_cm:.2f} cm")

# --- Display image with annotations ---
plt.imshow(img_rgb)
plt.title(f"Estimated food height: {food_height_cm:.2f} cm")
plt.axis('off')
plt.show()




Using cache found in /root/.cache/torch/hub/intel-isl_MiDaS_master
Using cache found in /root/.cache/torch/hub/intel-isl_MiDaS_master
Using cache found in /root/.cache/torch/hub/ultralytics_yolov5_master
YOLOv5 🚀 2025-8-8 Python-3.11.13 torch-2.6.0+cu124 CPU

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients, 16.4 GFLOPs
Adding AutoShape... 


Saving spaghetti-plate-1-1321302.jpg to spaghetti-plate-1-1321302 (1).jpg


/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:906: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):


Spoon pixel length: 888.16
Estimated food height: 0.25 cm


In [ ]:
# Convert to grayscale
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

# Simple threshold to create binary mask (tweak values!)
_, mask = cv2.threshold(gray, 120, 255, cv2.THRESH_BINARY_INV)

# Calculate area (assuming mask is binary: 0 and 255)
food_area = np.sum(mask == 255)

# Show mask
plt.imshow(mask, cmap='gray')
plt.title(f"Food area: {food_area} pixels")
plt.axis('off')
plt.show()

NameError: name 'cv2' is not defined